<a href="https://colab.research.google.com/github/pymoux/wildfires_DST/blob/Xavier/wildfires_datat_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries and data

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import geopandas as gpd
from shapely.geometry import Point


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
db_file_15 = "/content/drive/MyDrive/DST_Wildfires/2015/FPA_FOD_20170508.sqlite"

con = sqlite3.connect(db_file_15)
df_wild = pd.read_sql_query("SELECT * from Fires", con, index_col='OBJECTID')

con.close()


In [ ]:
df_wild.head(10)


,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,1,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,13,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,27,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,43,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,44,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...
6,6,FS-1418849,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,54,...,A,38.635278,-120.103611,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00\xf0<~\x90\xa1\x06^\...
7,7,FS-1418851,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,58,...,A,38.688333,-120.153333,5.0,USFS,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00$o\x996\xd0\t^\xc0h\...
8,8,FS-1418854,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,3,...,B,40.968056,-122.433889,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00t)\xe8\xd5\xc4\x9b^\...
9,9,FS-1418856,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,5,...,B,41.233611,-122.283333,13.0,STATE OR PRIVATE,CA,None,None,None,"b'\x00\x01\xad\x10\x00\x00\xdc\x8d\x1e""""\x92^\..."


## Remove useless columns

In [ ]:
# select columns of interest for the analysis
df_interest = df_wild[["FOD_ID",
                       "FPA_ID",
                       "FIRE_YEAR",
                       "DISCOVERY_DATE",
                       "DISCOVERY_DOY",
                       "DISCOVERY_TIME",
                       "STAT_CAUSE_CODE",
                       "STAT_CAUSE_DESCR",
                       "CONT_DATE",
                       "CONT_DOY",
                       "CONT_TIME",
                       "FIRE_SIZE",
                       "FIRE_SIZE_CLASS",
                       "LATITUDE",
                       "LONGITUDE",
                       "STATE",
                       "COUNTY",
                       "FIPS_CODE",
                       "FIPS_NAME"]]
df_interest.head()


,FOD_ID,FPA_ID,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
OBJECTID,,,,,,,,,,,,,,,,,,,
1,1,FS-1418826,2005,2453403.5,33,1300,9.0,Miscellaneous,2453403.5,33.0,1730,0.10,A,40.036944,-121.005833,CA,63,063,Plumas
2,2,FS-1418827,2004,2453137.5,133,0845,1.0,Lightning,2453137.5,133.0,1530,0.25,A,38.933056,-120.404444,CA,61,061,Placer
3,3,FS-1418835,2004,2453156.5,152,1921,5.0,Debris Burning,2453156.5,152.0,2024,0.10,A,38.984167,-120.735556,CA,17,017,El Dorado
4,4,FS-1418845,2004,2453184.5,180,1600,1.0,Lightning,2453189.5,185.0,1400,0.10,A,38.559167,-119.913333,CA,3,003,Alpine
5,5,FS-1418847,2004,2453184.5,180,1600,1.0,Lightning,2453189.5,185.0,1200,0.10,A,38.559167,-119.933056,CA,3,003,Alpine


In [ ]:
# drop useless columns
df_interest_2 = df_wild.drop(columns=['SOURCE_SYSTEM_TYPE',
                                      'SOURCE_SYSTEM',
                                      'NWCG_REPORTING_AGENCY',
                                      'NWCG_REPORTING_UNIT_ID',
                                      'NWCG_REPORTING_UNIT_NAME',
                                      'SOURCE_REPORTING_UNIT',
                                      'SOURCE_REPORTING_UNIT_NAME',
                                      'LOCAL_FIRE_REPORT_ID',
                                      'LOCAL_INCIDENT_ID',
                                      'FIRE_CODE',
                                      'FIRE_NAME',
                                      'ICS_209_INCIDENT_NUMBER',
                                      'ICS_209_NAME',
                                      'MTBS_ID',
                                      'MTBS_FIRE_NAME',
                                      'COMPLEX_NAME',
                                      'OWNER_CODE',
                                      'OWNER_DESCR',
                                      'Shape'])


## Transform dates

#### Discovery date

In [ ]:
# discovery date from YEAR + DoY into Gregorian date format
df_interest["DISCO_DATE"] = df_interest["FIRE_YEAR"]*1000 + df_interest["DISCOVERY_DOY"]
df_interest["DISCO_DATE"] = pd.to_datetime(df_interest["DISCO_DATE"], format = "%Y%j")

df_interest[["DISCOVERY_DATE", "FIRE_YEAR", "DISCOVERY_DOY", "DISCO_DATE"]].head()


<ipython-input-13-cfadd650c7b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interest["DISCO_DATE"] = df_interest.apply(lambda x: x["FIRE_YEAR"]*1000 + x["DISCOVERY_DOY"], axis=1)


,DISCOVERY_DATE,FIRE_YEAR,DISCOVERY_DOY,DISCO_DATE
OBJECTID,,,,
1,2453403.5,2005,33,2005033
2,2453137.5,2004,133,2004133
3,2453156.5,2004,152,2004152
4,2453184.5,2004,180,2004180
5,2453184.5,2004,180,2004180


In [ ]:
# other ways by converting julian date
df_interest["DISCO_DATE_2"] = pd.to_datetime(df_interest["DISCOVERY_DATE"], origin="julian", unit="D")
df_interest["DISCO_DATE_3"] = pd.to_datetime(df_interest["DISCOVERY_DATE"] - pd.Timestamp(0).to_julian_date(), unit="D")

df_interest[["DISCOVERY_DATE", "FIRE_YEAR", "DISCOVERY_DOY", "DISCO_DATE", "DISCO_DATE_2", "DISCO_DATE_3"]].head()


In [ ]:
# check 3 discovery dates
df_interest['disco_matching'] = df_interest.apply(lambda x: x.DISCO_DATE == x.DISCO_DATE_2 == x.DISCO_DATE_3, axis = 1)
#df_interest['all_matching'].value_counts()
df_interest[['FOD_ID', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCO_DATE', 'DISCO_DATE_2', 'DISCO_DATE_3', 'disco_matching']].loc[df_interest['all_matching']==False]

# 4 mismatches in FIRE_YEAR


#### Control date

In [ ]:
# control date from YEAR + DoY into Gregorian date format
df_interest["CONTROL_DATE"] = df_interest["FIRE_YEAR"]*1000 + df_interest["CONT_DOY"]
df_interest["CONTROL_DATE"] = pd.to_datetime(df_interest["CONTROL_DATE"], format = "%Y%j", errors='ignore')
#df_interest['CONTROL_DATE'].replace({np.nan:None}, inplace=True)

df_interest[["DISCOVERY_DATE", "FIRE_YEAR", "DISCO_DATE", "CONT_DOY", "CONTROL_DATE"]].sample(10)

<ipython-input-12-3312161aa816>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interest["CONTROL_DATE"] = df_interest["FIRE_YEAR"]*1000 + df_interest["CONT_DOY"]
<ipython-input-12-3312161aa816>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interest["CONTROL_DATE"] = pd.to_datetime(df_interest["CONTROL_DATE"], format = "%Y%j", errors='ignore')


,DISCOVERY_DATE,FIRE_YEAR,DISCO_DATE,CONT_DOY,CONTROL_DATE
OBJECTID,,,,,
1704263,2453072.5,2004,2004-03-08,NaN,NaT
768531,2452503.5,2002,2002-08-17,NaN,NaT
945172,2452367.5,2002,2002-04-03,NaN,NaT
187615,2450278.5,1996,1996-07-14,197.0,1996-07-15
92836,2450156.5,1996,1996-03-14,74.0,1996-03-14
720008,2451699.5,2000,2000-06-04,156.0,2000-06-04
582352,2454526.5,2008,2008-03-01,NaN,NaT
1712947,2454129.5,2007,2007-01-29,NaN,NaT
320496,2453891.5,2006,2006-06-05,156.0,2006-06-05


In [ ]:
df_interest['test_date'] = np.where(df_interest['CONTROL_DATE'] < df_interest['DISCO_DATE'], True, False)
#df_interest['CONTROL_DATEx'] = pd.to_datetime(df_interest['CONTROL_DATEx'], format = "%Y%j")
df_interest['CONTROL_DATEx'] = np.where(df_interest['test_date'], df_interest['CONTROL_DATE'] + pd.offsets.DateOffset(years=1), df_interest['CONTROL_DATE'])
df_interest.loc[df_interest['CONTROL_DATE'] != df_interest['CONTROL_DATEx']]


<ipython-input-34-839beaa778cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interest['test_date'] = np.where(df_interest['CONTROL_DATE'] < df_interest['DISCO_DATE'], True, False)
<ipython-input-34-839beaa778cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interest['CONTROL_DATEx'] = np.where(df_interest['test_date'], df_interest['CONTROL_DATE'] + pd.offsets.DateOffset(years=1), df_interest['CONTROL_DATE'])


,FOD_ID,FPA_ID,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,...,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME,DISCO_DATE,CONTROL_DATE,CONTROL_DATEx,test_date
OBJECTID,,,,,,,,,,,,,,,,,,,,,
85,86,FS-1419068,2005,2453555.5,185,0530,9.0,Miscellaneous,NaN,NaN,...,48.228056,-115.480278,MT,53,053,Lincoln,2005-07-04,NaT,NaT,False
1507,1521,FS-1422198,2005,2453440.5,70,1400,4.0,Campfire,NaN,NaN,...,37.029722,-92.125556,MO,67,067,Douglas,2005-03-11,NaT,NaT,False
1508,1522,FS-1422199,2005,2453440.5,70,1720,7.0,Arson,NaN,NaN,...,36.829722,-92.074167,MO,91,091,Howell,2005-03-11,NaT,NaT,False
1553,1567,FS-1422292,2005,2453444.5,74,1400,5.0,Debris Burning,NaN,NaN,...,36.671944,-92.599722,MO,153,153,Ozark,2005-03-15,NaT,NaT,False
1556,1570,FS-1422297,2005,2453445.5,75,1330,7.0,Arson,NaN,NaN,...,36.903889,-92.980556,MO,43,043,Christian,2005-03-16,NaT,NaT,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880455,300348310,2015CAIRS28424493,2015,2457277.5,255,0638,13.0,Missing/Undefined,NaN,NaN,...,38.048577,-120.313090,CA,None,None,None,2015-09-12,NaT,NaT,False
1880462,300348373,2015CAIRS29217935,2015,2457300.5,278,0126,9.0,Miscellaneous,NaN,NaN,...,37.617619,-120.938570,CA,None,None,None,2015-10-05,NaT,NaT,False
1880463,300348375,2015CAIRS28364460,2015,2457144.5,122,2052,13.0,Missing/Undefined,NaN,NaN,...,37.617619,-120.938570,CA,None,None,None,2015-05-02,NaT,NaT,False


In [ ]:
df_interest["CONTROL_DATE_2"] = pd.to_datetime(df_interest["CONT_DATE"], origin="julian", unit="D")
df_interest["CONTROL_DATE_3"] = pd.to_datetime(df_interest["CONT_DATE"] - pd.Timestamp(0).to_julian_date(), unit="D")

df_interest[["DISCOVERY_DATE", "FIRE_YEAR", "CONT_DOY", "CONTROL_DATE", "CONTROL_DATE_2", "CONTROL_DATE_3"]].head()

In [ ]:
df_interest['CONTROL_DATE'].replace({np.nan:None}, inplace=True)
df_interest['CONTROL_DATE_2'].replace({np.nan:None}, inplace=True)
df_interest['CONTROL_DATE_3'].replace({np.nan:None}, inplace=True)
df_interest.head()

In [ ]:
# check 3 control dates
df_interest['cont_matching'] = df_interest.apply(lambda x: x.CONTROL_DATE == x.CONTROL_DATE_2 == x.CONTROL_DATE_3, axis = 1)
#df_interest['all_matching'].value_counts()
df_interest[['FOD_ID', 'FIRE_YEAR', 'CONT_DATE', 'CONT_DOY', 'CONTROL_DATE', 'CONTROL_DATE_2', 'CONTROL_DATE_3', 'cont_matching']].loc[df_interest['cont_matching']==False]

# 4 mismatches in FIRE_YEAR

#### add DURATION column

In [ ]:
df_interest['FIRE_DURATION'] = (df_interest['CONTROL_DATE'] - df_interest['DISCO_DATE']).dt.days
df_interest['FIRE_DURATION'].unique()

<ipython-input-15-6e0f5128b3ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interest['FIRE_DURATION'] = (df_interest['CONTROL_DATE'] - df_interest['DISCO_DATE']).dt.days


array([   0.,    5.,    1.,   15.,    4.,   nan,    3.,    2.,    8.,
         10.,    6.,   24.,   26.,   13.,    9.,    7.,   22.,   11.,
         20.,   12.,   19.,   17.,   92.,   16.,   28.,   14.,   23.,
         31.,   45.,   38.,   32.,   46.,   29.,   27.,   65.,   50.,
         56.,   25.,   66.,   44.,   81.,   59.,   58.,   57.,   55.,
         53.,   52.,   51.,   91.,   89.,  126.,   93.,   35.,   30.,
         34.,   70.,   47.,   21.,   95.,  124.,   76.,   36.,   90.,
         54.,   60.,   48.,   73.,   18.,   85.,   63.,   84.,   33.,
         79.,   78.,   75.,   88.,   40.,   41.,   87.,   83.,  144.,
        133.,   94.,   64.,   77.,  139.,   42.,   96.,   37.,   82.,
         80.,   67.,  128.,  150.,  104.,   39.,  109.,   43.,  102.,
        105.,  103.,   99.,   97.,   62.,  112.,  111.,   86.,  123.,
         49., -361., -363., -364.,  117.,   68.,   98.,   74.,  122.,
        108.,  129.,  148.,  116.,   69.,   72.,  155.,  141.,  137.,
        161.,   61.,

## Fill missing county values

In [ ]:
# define geometry points
gdf = gpd.GeoDataFrame(df_interest[['FOD_ID', 'LATITUDE', 'LONGITUDE', 'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME']], geometry=gpd.points_from_xy(df_interest['LONGITUDE'], df_interest['LATITUDE']))
gdf.head()


,FOD_ID,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME,geometry
OBJECTID,,,,,,,,
1,1,40.036944,-121.005833,CA,63,063,Plumas,POINT (-121.00583 40.03694)
2,2,38.933056,-120.404444,CA,61,061,Placer,POINT (-120.40444 38.93306)
3,3,38.984167,-120.735556,CA,17,017,El Dorado,POINT (-120.73556 38.98417)
4,4,38.559167,-119.913333,CA,3,003,Alpine,POINT (-119.91333 38.55917)
5,5,38.559167,-119.933056,CA,3,003,Alpine,POINT (-119.93306 38.55917)


In [ ]:
# load counties geometry from US census
gdf_counties = gpd.read_file("/content/drive/MyDrive/DST_Wildfires/geopandas/tl_2023_us_county.shx")
gdf_counties = gdf_counties[['GEOID', 'STATEFP', 'COUNTYFP', 'NAME', 'geometry']]
gdf_counties = gdf_counties.rename(columns={"NAME": "COUNTYNAME"})


In [ ]:
# load states geometry from US census
gdf_states = gpd.read_file("/content/drive/MyDrive/DST_Wildfires/geopandas/tl_2023_us_state.shx")
gdf_states = gdf_states[['STATEFP', 'STUSPS', 'NAME']]
gdf_states = gdf_states.rename(columns={"NAME": "STATENAME"})

# load states areas
states_area = pd.read_csv("/content/drive/MyDrive/DST_Wildfires/geopandas/states_area.tsv",
                          sep='\t',
                          skiprows=2,
                          usecols=['State', 'Total_area_SqMi', 'Land_area_SqMi', 'Total_water_area_SqMi'])
for col_sm, col_a in zip(['Total_area_SqMi', 'Land_area_SqMi', 'Total_water_area_SqMi'], ['Total_area_acres', 'Land_area_acres', 'Total_water_area_acres']):
  states_area[col_a] = states_area[col_sm].apply(lambda x: int(x.replace(',', '')) / 0.0015625)

gdf_states = pd.merge(left=gdf_states, right=states_area[['State', 'Total_area_acres', 'Land_area_acres',	'Total_water_area_acres']], left_on='STATENAME', right_on='State', how='left')
gdf_states.drop(columns='State', inplace=True)
#gdf_states = gdf_states.rename(columns={"NAME": "STATENAME"})
gdf_states.head()

,STATEFP,STUSPS,STATENAME,Total_area_acres,Land_area_acres,Total_water_area_acres
0,54,WV,West Virginia,15507200.0,15384320.0,122880.0
1,12,FL,Florida,42085120.0,34320000.0,7765120.0
2,17,IL,Illinois,37064960.0,35532160.0,1532800.0
3,27,MN,Minnesota,55639040.0,50961280.0,4677760.0
4,24,MD,Maryland,7939840.0,6212480.0,1727360.0


In [ ]:
# merge states gdf to counties gdf
gdf_counties = pd.merge(left=gdf_counties, right=gdf_states[['STATEFP', 'STUSPS', 'STATENAME']], on='STATEFP')

In [ ]:
# identify missing counties by joining on geometry
joined_gdf = gpd.sjoin(gdf, gdf_counties, how='left', op='within')
#new_df = new_df.rename(columns={'NAME': 'new_counties'})
joined_gdf.drop(columns='index_right', inplace=True)
joined_gdf.head(10)


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


,FOD_ID,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME,geometry,GEOID,STATEFP,COUNTYFP,COUNTYNAME,STUSPS,STATENAME
OBJECTID,,,,,,,,,,,,,,
1,1,40.036944,-121.005833,CA,63,063,Plumas,POINT (-121.00583 40.03694),06063,06,063,Plumas,CA,California
2,2,38.933056,-120.404444,CA,61,061,Placer,POINT (-120.40444 38.93306),06017,06,017,El Dorado,CA,California
3,3,38.984167,-120.735556,CA,17,017,El Dorado,POINT (-120.73556 38.98417),06017,06,017,El Dorado,CA,California
4,4,38.559167,-119.913333,CA,3,003,Alpine,POINT (-119.91333 38.55917),06003,06,003,Alpine,CA,California
5,5,38.559167,-119.933056,CA,3,003,Alpine,POINT (-119.93306 38.55917),06003,06,003,Alpine,CA,California
6,6,38.635278,-120.103611,CA,5,005,Amador,POINT (-120.10361 38.63528),06005,06,005,Amador,CA,California
7,7,38.688333,-120.153333,CA,17,017,El Dorado,POINT (-120.15333 38.68833),06017,06,017,El Dorado,CA,California
8,8,40.968056,-122.433889,CA,None,None,None,POINT (-122.43389 40.96806),06089,06,089,Shasta,CA,California
9,9,41.233611,-122.283333,CA,None,None,None,POINT (-122.28333 41.23361),06093,06,093,Siskiyou,CA,California


In [ ]:
# merge new location to dataframe
df_interest = pd.merge(df_interest, joined_gdf[['FOD_ID', 'GEOID', 'STATEFP', 'COUNTYFP', 'COUNTYNAME', 'STUSPS', 'STATENAME']], on='FOD_ID')
df_interest.head()


,FOD_ID,FPA_ID,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,...,FIPS_NAME,DISCO_DATE_1,DISCO_DATE_2,DISCO_DATE_3,GEOID,STATEFP,COUNTYFP,COUNTYNAME,STUSPS,STATENAME
0,1,FS-1418826,2005,2453403.5,33,1300,9.0,Miscellaneous,2453403.5,33.0,...,Plumas,2005-02-02,2005-02-02,2005-02-02,06063,06,063,Plumas,CA,California
1,2,FS-1418827,2004,2453137.5,133,0845,1.0,Lightning,2453137.5,133.0,...,Placer,2004-05-12,2004-05-12,2004-05-12,06017,06,017,El Dorado,CA,California
2,3,FS-1418835,2004,2453156.5,152,1921,5.0,Debris Burning,2453156.5,152.0,...,El Dorado,2004-05-31,2004-05-31,2004-05-31,06017,06,017,El Dorado,CA,California
3,4,FS-1418845,2004,2453184.5,180,1600,1.0,Lightning,2453189.5,185.0,...,Alpine,2004-06-28,2004-06-28,2004-06-28,06003,06,003,Alpine,CA,California
4,5,FS-1418847,2004,2453184.5,180,1600,1.0,Lightning,2453189.5,185.0,...,Alpine,2004-06-28,2004-06-28,2004-06-28,06003,06,003,Alpine,CA,California


In [ ]:
# wilfires without any location
nana_index = df_interest[(df_interest['FIPS_CODE'].isna()) & (df_interest['COUNTYFP'].isna())].index
df_interest.loc[nana_index, ['STATE','LATITUDE','LONGITUDE']]

#TO COMPLETE MANUALLY?

,STATE,LATITUDE,LONGITUDE
308961,AZ,32.691400,-114.734400
308996,AZ,32.700000,-114.730000
1063465,AZ,31.951100,-113.051900
1065586,AZ,32.592000,-114.799700
1650995,TX,26.066667,-97.980278


In [ ]:
# wilfires without new location
na_index = df_interest[(df_interest['COUNTYFP'].isna())].index
df_interest.loc[na_index, ['STATE', 'FIPS_NAME', 'LATITUDE','LONGITUDE']]


,STATE,FIPS_NAME,LATITUDE,LONGITUDE
24104,AZ,Santa Cruz,31.333056,-110.639444
25730,AZ,Cochise,31.333333,-110.320833
156246,AZ,Cochise,31.333056,-110.390278
172124,AZ,Santa Cruz,31.332500,-110.983056
308885,AZ,Yuma,32.566400,-114.792700
308961,AZ,None,32.691400,-114.734400
308996,AZ,None,32.700000,-114.730000
309006,AZ,Yuma,32.512000,-114.803000
714048,ME,Aroostook,47.183334,-68.883331
1063465,AZ,None,31.951100,-113.051900


In [ ]:
# wilfires with divergent new location
div_index = df_interest[df_interest['FIPS_NAME'] != df_interest['COUNTYNAME']].index
df_interest.loc[na_index, ['STATE', 'FIPS_NAME', 'LATITUDE', 'LONGITUDE', 'COUNTYNAME']]

,STATE,FIPS_NAME,LATITUDE,LONGITUDE,COUNTYNAME
24104,AZ,Santa Cruz,31.333056,-110.639444,NaN
25730,AZ,Cochise,31.333333,-110.320833,NaN
156246,AZ,Cochise,31.333056,-110.390278,NaN
172124,AZ,Santa Cruz,31.332500,-110.983056,NaN
308885,AZ,Yuma,32.566400,-114.792700,NaN
308961,AZ,None,32.691400,-114.734400,NaN
308996,AZ,None,32.700000,-114.730000,NaN
309006,AZ,Yuma,32.512000,-114.803000,NaN
714048,ME,Aroostook,47.183334,-68.883331,NaN
1063465,AZ,None,31.951100,-113.051900,NaN


## Step to be defined

## Export ready-to-use dataset

In [ ]:
df_interest.to_csv('/content/drive/MyDrive/DST_Wildfires/db_output/wildfires_db.csv', sep=',')
